### Setup

In [ ]:
!git clone https://github.com/rohaldb/NSFF-custom
!mv NSFF-custom Neural-Scene-Flow-Fields

In [ ]:
%cd /content/Neural-Scene-Flow-Fields
!pip install -r requirements.txt

In [ ]:
#download data
%cd /content/
!gdown https://drive.google.com/uc?id=1wm7D68vDBctDwsTLfevvZM5wEiIp9cLR
!unzip "custom_nerf_data.zip"; rm "custom_nerf_data.zip";
#add nsff-sf estimates
%cd /content/nerf_data/kid-running/dense
!gdown https://drive.google.com/uc?id=1mp-ikirWScLSwsaCFRLoHZfSMqA9lR7v
!unzip "sf_nsff.zip"; rm "sf_nsff.zip"

### Training a new model
Unfortunately there is no good way to download the images off tensorboard. So best way to do it is just to drag them to desktop and upload to google drive

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/Neural-Scene-Flow-Fields/nsff_exp/logs/summaries

In [11]:
txt = """
expname = kid-running_ndc_5f_sv_of_sm_unify3_testing-{}

basedir = ./logs
datadir = /content/nerf_data/kid-running/dense

dataset_type = llff

factor = 2
llffhold = 10

N_rand = 1024
N_samples = 128
N_importance = 0
netwidth = 256

use_viewdirs = True
raw_noise_std = 1e0
no_ndc = False
lindisp = False
no_batching = True
spherify = False
decay_depth_w = True
decay_optical_flow_w = True
use_motion_mask = True
num_extra_sample = 512
decay_iteration = 3

w_depth = 0.04
w_optical_flow = 0.02
w_sm = 0.1
w_sf_reg = 0.01
w_cycle = 1.0
w_prob_reg = 0.1

start_frame = 8
end_frame = 11
"""

In [ ]:
%cd /content/Neural-Scene-Flow-Fields/nsff_exp
for j in [10]:
  with open("/content/Neural-Scene-Flow-Fields/nsff_exp/configs/config_kid-running.txt", "w") as text_file:
    text_file.write(txt.format(j))
  !python run_nerf.py --config configs/config_kid-running.txt --target_idx 1 --i_weights 2000 --i_img 2500 --i_vid 2500 --i_print 100 --w_sf {j} --chunk {1024*16} --netchunk {1024*16} 

In [ ]:
while True:
  pass

In [ ]:
#copy results to google drive
%cp -R /content/Neural-Scene-Flow-Fields/nsff_exp/logs/* "/content/drive/MyDrive/Thesis/sf reg/motion accounted for sf decay/"
%ls "/content/drive/MyDrive/Thesis/sf reg/motion accounted for/"

###Rendering from my pretrained model

In [ ]:
!rm -rf /content/Neural-Scene-Flow-Fields/nsff_exp/logs

In [ ]:
%cd /content/Neural-Scene-Flow-Fields/nsff_exp
!mkdir -p "./logs/"
%cp -R "/content/drive/MyDrive/Thesis/sf reg/motion accounted for/." ./logs

In [ ]:
%%time
!python run_nerf.py --config configs/config_kid-running.txt --target_idx 1 --render_single_frame

In [ ]:
%%time
!python run_nerf.py --config configs/config_kid-running.txt --target_idx 1 --render_lockcam_slowmo

In [ ]:
%%time
!python run_nerf.py --config configs/config_kid-running.txt --render_bt --bt_linear_interpolation --target_idx 1